
# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur adjoint au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>

---


# TD 9 : Théorie de l'information
___

### Description

Dans ce travail dirigé, vous aurez l'occasion de déterminer quels mots sont les plus discriminants pour la construction du filtre anti-spam du TD8.

L'exercice 1 vous permettra de programmer les fonctions utiles pour classer les mots en fonction de leur information mutuelle avec la catégorie du message (courriel ou pourriel). L'exercice 2 vous permettra d'estimer la loi conjointe et les lois marginales entre une variable explicative $X_j$ et $Y$ avec les messages électroniques obtenus. Dans l'exercice 3, vous identifierez les mots les plus discriminant pour classer les messages électroniques en courriel ou pourriel.

### Données

Les données exploitées dans ce TD correspondent à des messages électroniques authentiques d'un employé de la compagnie Enron. Vous pouvez télécharger le jeux de données à partir du site web du cours. Le fichier doit être décompressé dans un dossier nommé *data* du répertoire courant de votre calepin Jupyter. 

Ces données ont déjà été utilisées dans le TD8. 


In [1]:
using Statistics

## Préliminaires

Cette section permet de dénombrer le nombre de courriels et de pourriels contenant chacun des mots répertoriés. Les deux dictionnaires suivants sont retournés :
- ham_wordcounts
- spam_word_counts

In [2]:
"""
wordlisting : Cette fonction transforme un fichier texte en une liste de mots. Le nombre de fois que
              les mots apparaîssent ne sont pas répertorié.
Input: filename::String le chemin du fichier texte 
Output: Array{Sting} Liste des mots contenus dans le fichier texte
"""

function wordlisting(filename::String)
    
    f = read(filename, String)
    text = replace(f, r"[0123456789]" => "")
    words = split(text, r"\W+")
    filter!(x -> length(x) > 1, words)
    wordlist = unique(words)
    
end

"""
wordcounting : À partir d'un Array of Array contenant la liste des mots d'un ensemble de fichiers texte, la fonction 
               retourne le nombre de fois où chaque mot est présent.
Input: Un array correspondant à la liste des mots pour chacun des fichiers texte 
Output: Dictionnaire compilant le nombre de ligne dans lesquelles chacun des mots apparaît.
"""

function wordcounting(A::Array{Array{SubString{String},1},1})
   
    words = vcat(A...)

    wordcounts = Dict{String,Int64}()

    for word in words
        wordcounts[word]=get(wordcounts, word, 0) + 1
    end
    
    return wordcounts
    
end


# Récupération des noms de fichier de tous les hams
filesdir = "data/enron1/ham/"
filename_ham = filesdir.*readdir(filesdir)

# Récupération des noms de fichier de tous les spams
filesdir = "data/enron1/spam/"
filename_spam = filesdir.*readdir(filesdir)

ham_wordlist = wordlisting.(filename_ham)
ham_wordcounts = wordcounting(ham_wordlist)

spam_wordlist = wordlisting.(filename_spam)
spam_wordcounts = wordcounting(spam_wordlist);



## Exercice 1

Programmation des fonctions utiles en théorie de l'information.

### (a) Entropie 

Programmez une fonction permettant de calculer l'entropie d'une variable aléatoire catégorielle $Y$ ayant comme argument le vecteur de probabilités des résulats possibles de $Y$.

Testez votre fonction de la façon suivante :

`p = [1/4,1/4,1/4,1/4]`

`entropy(p)`

In [3]:
function entropy(p::Vector{<:Real})
    
    @assert isapprox(sum(p),1)
    @assert all( 0 .<= p .<= 1 )
    
    H = 0.0
    
    for i in eachindex(p)
    
        if !isapprox(p[i],0)
            H += -p[i] * log2(p[i])
        end
        
    end
        
    return H
    
end

entropy (generic function with 1 method)

In [4]:
p = [1/4,1/4,1/4,1/4]

entropy(p)

2.0

### (b) Entropie croisée

Programmez une fonction permettant de calculer l'entropie croisée d'une variable aléatoire catégorielle $Y$ ayant comme argument sont vecteur de probabilités des résulats possibles et comme deuxième argument la second vecteur de probabité correspondant à une autre fonction de masse pour $Y.

Testez votre fonction de la façon suivante :

`p = [1/2,1/4,1/8,1/8]`

`q = [1/4,1/4,1/4,1/4]`

`crossentropy(p,q)`

In [5]:
function crossentropy(p::Vector{<:Real},q::Vector{<:Real})
    
    @assert length(p) == length(q)
    @assert isapprox(sum(p),1)
    @assert all( 0 .<= p .<= 1 )
    @assert isapprox(sum(q),1)
    @assert all( 0 .<= q .<= 1 )
        
    H = 0.0
    
    for i in eachindex(p)
    
        H += -p[i] * log2(q[i])
        
    end
        
    return H
    
end

crossentropy (generic function with 1 method)

In [6]:
p = [1/2,1/4,1/8,1/8]
q = [1/4,1/4,1/4,1/4]
crossentropy(p,q)

2.0

### (c) Divergence de Kullback-Leibler

Programmez une fonction permettant de calculer la divergence de Kullback-Leibler entre les deux fonctions de masse caractérisées par les vecteurs de probabilités $p$ et $q$.

Testez votre fonction de la façon suivante :

`p = [1/2,1/4,1/8,1/8]`

`q = [1/4,1/4,1/4,1/4]`

`kullback(p,q)`

In [7]:
function kullback(p::Vector{<:Real}, q::Vector{<:Real})
       
    kl = -entropy(p) + crossentropy(p,q)
    
    return kl
    
end

kullback (generic function with 1 method)

In [8]:
p = [1/2,1/4,1/8,1/8]
q = [1/4,1/4,1/4,1/4]
kullback(p,q)

0.25

## Exercice 2 

Estimation des lois conjointes et marginales des variables $X$ et $Y$ avec le jeu de données.

### (a) Soit le mot *enron*, construisez le tableau des fréquences observées de la variable $X$ et $Y$.

Ce tableau des fréquences observées est illustré à l'exemple 8 du Chapitre 9 des notes de cours. Servez vous des dictionnaires ham_wordcounts et spam_wordcounts.

In [9]:
mot = "enron"

n₀ = length(filename_spam)
n₁ = length(filename_ham)
n = n₀ + n₁

if haskey(spam_wordcounts, mot)
    n₀₁ = spam_wordcounts[mot]
else
    n₀₁ = 0
end

if haskey(ham_wordcounts,mot)
    n₁₁ = ham_wordcounts[mot]
else
    n₁₁ = 0
end

T = [n₀-n₀₁ n₁-n₁₁ ; n₀₁ n₁₁]

2×2 Array{Int64,2}:
 1500  2210
    0  1462

### (b) Avec le tableau des fréquences observées, estimez la loi de probabilité conjointe de $X$ et $Y$.

Produisez un vecteur de probabilités représentant les 4 possibilités.

In [10]:
p = T/n

2×2 Array{Float64,2}:
 0.290023  0.427301
 0.0       0.282676

### (c) Avec le tableau des fréquences observées, estimez la loi de probabilité conjointe de $X$ et $Y$ estimée à l'aide du produit des lois marginales.

Produisez un vecteur de probabilités représentant les 4 possibilités.

In [11]:
p_X = sum(T,dims=1)/n
p_Y = sum(T,dims=2)/n
q = p_Y * p_X

2×2 Array{Float64,2}:
 0.208041   0.509283
 0.0819826  0.200693

### (d) Estimez l'information mutuelle entre $X$ et $Y$.

Utilisez la loi conjointe estimée et le produit des lois marginales estimées dans la divergence de Kullback-Leibler.

In [12]:
p = vec(p)
q = vec(q)
kullback(p,q)

0.17049542401082207

## Exercice 3

Identification des mots les plus discriminants.

### (a) En reprenant la méthode de l'exercice 2, calculez l'information mutuelle pour tous les mots contenus dans les messages.

In [13]:
function minfo(mot::String)
    n₀ = length(filename_spam)
    n₁ = length(filename_ham)
    n = n₀ + n₁

    if haskey(spam_wordcounts, mot)
        n₀₁ = spam_wordcounts[mot]
    else
        n₀₁ = 0
    end

    if haskey(ham_wordcounts,mot)
        n₁₁ = ham_wordcounts[mot]
    else
        n₁₁ = 0
    end

    T = [n₀-n₀₁ n₁-n₁₁ ; n₀₁ n₁₁]
    
    p = T/n
    
    p_X = sum(T,dims=1)/n
    p_Y = sum(T,dims=2)/n
    q = p_Y * p_X
    
    p = vec(p)
    q = vec(q)
    
    mutualInfo = kullback(p,q)
    
    return mutualInfo
    
end

minfo (generic function with 1 method)

In [14]:
liste_mot = unique(union(collect(keys(ham_wordcounts)), collect(keys(spam_wordcounts))))

mutualInfo = Float64[]

for mot in liste_mot

    push!(mutualInfo, minfo(mot))
    
end

### (b) Selon l'information mutuelle, quels sont les 10 mots les plus dépendants de $Y$ ?

In [15]:
ind = sortperm(mutualInfo, rev=true)
mot_discriminant = liste_mot[ind]
mot_discriminant[1:10]

10-element Array{String,1}:
 "Subject"  
 "enron"    
 "cc"       
 "hpl"      
 "daren"    
 "http"     
 "gas"      
 "forwarded"
 "pm"       
 "ect"      